<a href="https://colab.research.google.com/github/koh-hongQ/KMS_NIMS_AI_Camp/blob/main/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convolutional Neural Network

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms


In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

MNIST_training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
    )

MNIST_test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

MNIST_training_loader = DataLoader(
    MNIST_training_data,
    batch_size=64,
    shuffle=True
)

MNIST_test_loader = DataLoader(
    MNIST_test_data,
    batch_size=64,
    shuffle=False
)

In [ ]:
import torch.nn as nn

x, y = next(iter(MNIST_test_loader))
conv1 = nn.Conv2d(1, 6, 5)
print(x.shape)
x = conv1(x)
print(x.shape)

conv2 = nn.Conv2d(6, 16, 5, padding =2)
x = conv2(x)
print(x.shape)

pool = nn.MaxPool2d(2, 2)
x = pool(x)
print(x.shape)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def train_epoch(model, dataloader, loss_function, optimizer):
    train_loss = 0
    num_batches = len(dataloader)
    model.train()
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    model.to(device)
    for i, (x,y) in enumerate(dataloader):
        x = x.to(device)
        y = y.to(device)
        y_hat = model(x)
        loss = loss_function(y_hat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
        if (i+1)%1000 == 0:
            print(f"Batch: {i+1}")
    train_loss/= num_batches
    print(f"Train Error: Avg loss: {train_loss}")
    return train_loss

def test_epoch(model, dataloader, loss_function):
    model.eval()
    test_loss, correct = 0, 0
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    model.to(device)
    with torch.no_grad():
        for x,y in dataloader:
            x = x.to(device)
            y = y.to(device)
            y_hat = model(x)
            loss = loss_function(y_hat, y)
            test_loss += loss.item()
            correct += (y_hat.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: Accuracy: {(100*correct)}%, Avg loss: {test_loss}")
    return test_loss, correct

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()
train_loss_list = []
test_loss_list = []
for epoch in range(5):
    train_loss = train_epoch(net, MNIST_training_loader, loss_function, optimizer)
    test_loss, correct = test_epoch(net, MNIST_test_loader, loss_function)
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)

In [ ]:
plt.plot(range(len(train_loss_list)), train_loss_list, label = 'train loss')
plt.plot(range(len(test_loss_list)), test_loss_list, label = 'test loss')
plt.legend()
plt.show()

Finetuning

In [ ]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

cifar_trainingset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
cifar_trainingloader = DataLoader(cifar_trainingset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

cifar_testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
cifar_testloader = DataLoader(cifar_testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print(len(cifar_trainingset))
print(len(cifar_testset))

In [ ]:
x,y = cifar_trainingset[0]
x2 = x.permute(1,2,0)
print(classes[y])
plt.imshow(x2 /2  +0.5)
plt.show()

사전 정의된 모델들

In [ ]:
import torchvision
from torchvision.models import resnet18
re18 = resnet18()
x = torch.randn(16, 3, 32, 32)
re18(x).shape

In [ ]:
import torch
model_scratch = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
model_scratch(x).shape

In [ ]:
model_scratch

In [ ]:
model_scratch.layer1

In [ ]:
model_scratch.fc

In [ ]:
model_scratch.fc = nn.Linear(512, 10)
print(model_scratch(x).shape)

In [ ]:
model_pretrained = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model_pretrained.fc = nn.Linear(512, 10)

실습: model_scratch와 model_pretrained를 각각 훈련하여 test_loss를 같은 graph에 plot하시오.

In [ ]:
optimizer = torch.optim.SGD(model_pretrained.parameters(), lr = 1e-3, weight_decay= 1e-2)
optimizer.step()

In [ ]:
optimizer = torch.optim.Adam(model_pretrained.parameters(), lr = 1e-3, betas = (0.9,0.999))
optimizer.step()

In [ ]:
optimizer = torch.optim.RMSprop(model_pretrained.parameters(), lr = 1e-3, alpha = 0.99)
optimizer.step()

### Recurrent Neural Network

In [ ]:
import zipfile, os
with zipfile.ZipFile(os.path.join('.', 'data.zip'), 'r') as zip_ref:
    zip_ref.extractall('.')

os.listdir()

In [ ]:
import glob
data_dir = os.path.join('.', 'data', 'names')
text_files = glob.glob(os.path.join(data_dir, '*.txt'))
print(text_files)

In [ ]:

labels_set = set()
data = []
labels = []
for filename in text_files:
    label = os.path.splitext(os.path.basename(filename))[0]
    print(filename, label)
    labels_set.add(label)
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    for name in lines:
        data.append(name)
        labels.append(label)
print(data[:10])
print(labels[:10])

In [ ]:
import string
import unicodedata

allowed_characters = string.ascii_letters + " .,;'" + "_"
n_letters = len(allowed_characters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in allowed_characters
    )

print (f"converting 'Ślusàrski' to {unicodeToAscii('Ślusàrski')}")

In [ ]:
import torch
def letterToIndex(letter):
    # return our out-of-vocabulary character if we encounter a letter unknown to our model
    if letter not in allowed_characters:
        return allowed_characters.find("_")
    else:
        return allowed_characters.find(letter)

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), n_letters)
    for li, letter in enumerate(line):
        tensor[li][letterToIndex(letter)] = 1
    return tensor

In [ ]:
print (f"The letter 'a' becomes {lineToTensor('a')}") #notice that the first position in the tensor = 1
print (f"The name 'Ahn' becomes {lineToTensor('Ahn')}") #notice 'A' sets the 27th index to 1

In [ ]:
from io import open
import glob, os
import torch
from torch.utils.data import Dataset, DataLoader
class NamesDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir #for provenance of the dataset
        labels_set = set() #set of all classes

        self.data = []
        self.data_tensors = []
        self.labels = []
        self.labels_index = []
        text_files = glob.glob(os.path.join(data_dir, '*.txt'))
        for filename in text_files:
            label = os.path.splitext(os.path.basename(filename))[0]
            labels_set.add(label)
            lines = open(filename, encoding='utf-8').read().strip().split('\n')
            for name in lines:
                self.data.append(name)
                self.data_tensors.append(lineToTensor(name))
                self.labels.append(label)
        self.labels_uniq = list(labels_set)
        for idx in range(len(self.labels)):
            temp_index = self.labels_uniq.index(self.labels[idx])
            self.labels_index.append(temp_index)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_item = self.data[idx]
        data_label = self.labels[idx]
        data_tensor = self.data_tensors[idx]
        label_index = self.labels_index[idx]
        return data_tensor, label_index

In [ ]:
alldata = NamesDataset("data/names")
print(f"loaded {len(alldata)} items of data")
print(f"example = {alldata[0]}")

In [ ]:
train_set, test_set = torch.utils.data.random_split(alldata, [.85, .15])

print(f"train examples = {len(train_set)}, validation examples = {len(test_set)}")

train_dataloader = DataLoader(train_set, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=1, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()

        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.h2o = nn.Linear(hidden_size, output_size)

    def forward(self, line_tensor):
        rnn_out, hidden = self.rnn(line_tensor)
        output = self.h2o(hidden[0])
        return output

In [ ]:
x,y = nn.RNN(10, 128, batch_first=True)(torch.randn(32,6, 10))
x.shape, y.shape

In [ ]:
n_hidden = 128
rnn = CharRNN(n_letters, n_hidden, len(alldata.labels_uniq))
print(rnn)
x = torch.randn(1, 10, 58)
rnn(x).shape

In [ ]:
def label_from_output(output, output_labels):
    top_n, top_i = output.topk(1)
    label_i = top_i[0].item()
    return output_labels[label_i], label_i

input = lineToTensor('Albert').unsqueeze(0)
output = rnn(input)
print(output)
print(label_from_output(output, alldata.labels_uniq))

In [ ]:
import random
import numpy as np

def train_epoch(model, dataloader, loss_function, optimizer, every_iter = 1):
    train_loss = 0
    num_batches = len(dataloader)
    model.train()
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    model.to(device)
    for i, (x,y) in enumerate(dataloader):
        x = x.to(device)
        y = y.to(device)
        y_hat = model(x)
        loss = loss_function(y_hat, y)
        loss.backward()
        optimizer.step()
        nn.utils.clip_grad_norm_(model.parameters(), 3)     #clip gradient
        optimizer.zero_grad()
        train_loss += loss.item()
        if (i+1)%every_iter == 0:
            print(f"Batch: {i+1}, loss:{train_loss/i}")
    train_loss/= num_batches
    print(f"Train Error: Avg loss: {train_loss}")
    return train_loss
def test_epoch(model, dataloader, loss_function):
    model.eval()
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    model.to(device)
    test_loss, correct = 0, 0
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    with torch.no_grad():
        for x,y in dataloader:
            x = x.to(device)
            y = y.to(device)
            y_hat = model(x)
            loss = loss_function(y_hat, y)
            test_loss += loss.item()
            correct += (y_hat.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: Accuracy: {(100*correct)}%, Avg loss: {test_loss}")
    return test_loss, correct
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.03)
loss_function = nn.CrossEntropyLoss()

In [ ]:
train_loss_list = []
test_loss_list= []
for epoch in range(10):
    train_loss = train_epoch(rnn, train_dataloader, loss_function, optimizer, every_iter = 1000)
    test_loss, correct = test_epoch(rnn, test_dataloader, loss_function)
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss_list, label = 'train')
plt.plot(test_loss_list, label = 'test')
plt.legend()
plt.show()

연습문제: LSTM을 이용해 유사한 네트워크를 훈련하고 training loss와 test loss를 plot하라.
